# How to use the KServe Python SDK with ModelMesh

### This sample assumes ModelMesh Serving was deployed using the [quickstart guide](https://github.com/kserve/modelmesh-serving/blob/main/docs/quickstart.md).

In [1]:
from kubernetes import client
from kserve import constants
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1TFServingSpec
from kserve import V1beta1StorageSpec
from kserve import KServeClient

### The variables being set are `namespace`, `name`, `protocol_version`
- The `namespace` definition is where the InferenceService will be deployed to
- `name` will be the name of the `InferenceService`
- For ModelMesh, the [`v2`](https://github.com/kserve/modelmesh-serving/blob/main/docs/inference/ks-v2-grpc.md) protocol must be used since it doesn't support the default `v1` protocol

In [2]:
namespace = 'modelmesh-serving'
name='mnist-sample'
protocol_version='v2'

### Define InferenceService specifying ModelMesh as the deploymentMode

##### While both the KServe controller and ModelMesh controller will reconcile InferenceService resources, the ModelMesh controller will only handle those InferenceServices with the serving.kserve.io/deploymentMode: ModelMesh annotation. Otherwise, the KServe controller will handle reconciliation. The KServe controller will not reconcile an InferenceService with the serving.kserve.io/deploymentMode: ModelMesh annotation, and will defer under the assumption that the ModelMesh controller will handle it.

In [3]:
isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name, 
        namespace=namespace,
        annotations={
            'serving.kserve.io/deploymentMode': 'ModelMesh'
        }
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            sklearn=V1beta1SKLearnSpec(
                protocol_version=protocol_version,
                storage=V1beta1StorageSpec(
                    key='localMinIO',
                    path='sklearn/mnist-svm.joblib'
                )
            )
        )
    )
)

### Create InferenceService

In [4]:
kserve = KServeClient()
create = kserve.create(isvc)

In [19]:
print("name: ", create["metadata"]["name"])
print("uid: ", create["metadata"]["uid"])
print("key: ", create["spec"]["predictor"]["sklearn"]["storage"]["key"])
print("path: ", create["spec"]["predictor"]["sklearn"]["storage"]["path"])

name:  mnist-sample
uid:  910f307f-bf14-4f0a-b1dd-fb8fa9a9096f
key:  localMinIO
path:  sklearn/mnist-svm.joblib


### Check InferenceService status after deploying
#### notice how it is in a `Pending` state

In [6]:
pending_state = kserve.get(name, namespace=namespace)

In [7]:
print("name: ", pending_state["metadata"]["name"])
print("uid: ", pending_state["metadata"]["uid"])
print("status: ", pending_state["status"]["modelStatus"]["states"]["activeModelState"])

name:  mnist-sample
uid:  910f307f-bf14-4f0a-b1dd-fb8fa9a9096f
status:  Pending


### Check InferenceService status once its ready
#### State should now be `Loaded`

In [8]:
kserve.wait_isvc_ready(name, namespace=namespace)
loaded_state = kserve.get(name, namespace=namespace)

In [9]:
print("name: ", loaded_state["metadata"]["name"])
print("uid: ", loaded_state["metadata"]["uid"])
print("status: ", loaded_state["status"]["modelStatus"]["states"]["activeModelState"])

name:  mnist-sample
uid:  910f307f-bf14-4f0a-b1dd-fb8fa9a9096f
status:  Loaded


### Replace InferenceService and point it to a different model

In [10]:
updated_spec=V1beta1InferenceServiceSpec(
    predictor=V1beta1PredictorSpec(
        tensorflow=V1beta1TFServingSpec(
            protocol_version=protocol_version,
            storage=V1beta1StorageSpec(
                key='localMinIO',
                path='tensorflow/mnist.savedmodel'
                )
            )
        )
    )

In [11]:
updated_isvc = V1beta1InferenceService(api_version= constants.KSERVE_V1BETA1,
                          kind=constants.KSERVE_KIND,
                          metadata=client.V1ObjectMeta(name=name, namespace=namespace),
                          spec=updated_spec)

In [12]:
replace = kserve.replace(name, updated_isvc, namespace=namespace)

In [17]:
print("name: ", replace["metadata"]["name"])
print("uid: ", replace["metadata"]["uid"])
print("key: ", replace["spec"]["predictor"]["tensorflow"]["storage"]["key"])
print("path: ", replace["spec"]["predictor"]["tensorflow"]["storage"]["path"])


name:  mnist-sample
uid:  910f307f-bf14-4f0a-b1dd-fb8fa9a9096f
key:  localMinIO
path:  tensorflow/mnist.savedmodel


In [14]:
kserve.wait_isvc_ready(name, namespace=namespace)
updated = kserve.get(name, namespace=namespace)

In [15]:
print("name: ", updated["metadata"]["name"])
print("uid: ", updated["metadata"]["uid"])
print("status: ", updated["status"]["modelStatus"]["states"]["activeModelState"])

name:  mnist-sample
uid:  910f307f-bf14-4f0a-b1dd-fb8fa9a9096f
status:  Loaded


### Delete InferenceService

In [16]:
kserve.delete(name, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'mnist-sample',
  'group': 'serving.kserve.io',
  'kind': 'inferenceservices',
  'uid': '910f307f-bf14-4f0a-b1dd-fb8fa9a9096f'}}